In [ ]:
%cd /home/extend2/user7/Hwj/深度神经网络代码练习
!pwd

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optimizer
from torch.utils.data import Dataset,DataLoader 
from torchvision import transforms
from torchvision import datasets
import matplotlib.pyplot as plt

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
batch_size = 256
num_workers = 4
lr = 1e-2
epochs = 20

In [3]:
image_size = 28
data_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(image_size),
    transforms.ToTensor()
])

In [4]:
train_data = datasets.FashionMNIST(root='~/Hwj/深度神经网络代码练习',train=True,download=True,transform = data_transform)
test_data = datasets.FashionMNIST(root='~/Hwj/深度神经网络代码练习',train=False,download=True,transform = data_transform)



In [5]:
%cd /home/extend2/user7/Hwj/深度神经网络代码练习/FashionMNIST/raw

def convert(imgf, labelf, outf, n):
    f = open(imgf, "rb")
    o = open(outf, "w")
    l = open(labelf, "rb")

    f.read(16)
    l.read(8)
    images = []

    for i in range(n):
        image = [ord(l.read(1))]
        for j in range(28*28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image)+"\n")
    f.close()
    o.close()
    l.close()


if not os.path.exists('mnist_test.csv'):
    convert("train-images-idx3-ubyte", "train-labels-idx1-ubyte",
        "mnist_train.csv", 60000)
    convert("t10k-images-idx3-ubyte", "t10k-labels-idx1-ubyte",
            "mnist_test.csv", 10000)

/home/extend2/user7/Hwj/深度神经网络代码练习/FashionMNIST/raw


In [6]:
%cd /home/extend2/user7/Hwj/深度神经网络代码练习/FashionMNIST/raw

/home/extend2/user7/Hwj/深度神经网络代码练习/FashionMNIST/raw


In [7]:
class FMDataset(Dataset):
    def __init__(self,df,transform=None):
        self.df = df
        self.transform = transform
        self.images = df.iloc[:,1:].values.astype(np.uint8)
        self.label = df.iloc[:,0].values

    
    def __len__(self):
        return len(self.images)

    def __getitem__(self,idx):
        image = self.images[idx].reshape(28,28,1)
        label = int(self.label[idx])

        if self.transform is not None:
            image = self.transform(image)
        else:
            image = torch.tensor(image/255,dtype=torch.float)
        label = torch.tensor(label,dtype=torch.long)
        return image,label
        

train_df = pd.read_csv('~/Hwj/深度神经网络代码练习/FashionMNIST/raw/mnist_train.csv')
test_df = pd.read_csv('~/Hwj/深度神经网络代码练习/FashionMNIST/raw/mnist_test.csv')

train_data = FMDataset(train_df,data_transform)
test_data = FMDataset(test_df,data_transform)



In [8]:
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,num_workers=num_workers,drop_last=True)
test_loader = DataLoader(test_data,batch_size=batch_size,shuffle=True,num_workers=num_workers,drop_last=True)


In [9]:
image,label = next(iter(train_loader))
print(image.shape,label.shape)
plt.imshow(image[0][0],cmap='gray')


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/extend2/user7/anaconda3/envs/hwj_torch/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/extend2/user7/anaconda3/envs/hwj_torch/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/extend2/user7/anaconda3/envs/hwj_torch/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/extend2/user7/tmp/ipykernel_1172803/4222421904.py", line 17, in __getitem__
    image = self.transform(image)
  File "/home/extend2/user7/anaconda3/envs/hwj_torch/lib/python3.9/site-packages/torchvision/transforms/transforms.py", line 95, in __call__
    img = t(img)
  File "/home/extend2/user7/an

In [19]:
def initialize_weight(model):
    for m in model.modules():
        if isinstance(m,nn.Conv2d):
            torch.nn.init.normal_(m.weight.data)
            if m.bias is not None:
                torch.nn.init.constant_(m.bias.data,0.3)
        elif isinstance(m,nn.Linear):
            torch.nn.init.normal_(m.weight.data,0.1)
            if m.bias is not None:
                torch.nn.init.zeros_(m.bias.data)
        elif isinstance(m,nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()

In [21]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2),
            nn.Dropout(0.3),
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2),
            nn.Dropout(0.3)
        )

        self.fc = nn.Sequential(
            nn.Linear(64*4*4,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self,x):
        x = self.conv(x)
        x = x.view(-1,64*4*4)
        x = self.fc(x)
        return x
model= Net()
initialize_weight(model)
model =  model.cuda()



In [10]:
criterion = nn.CrossEntropyLoss()

In [11]:
optimizer = optimizer.Adam(model.parameters(),lr = 0.01)

NameError: name 'model' is not defined

In [24]:
def train(epoch):
    model.train()
    train_loss = 0
    for data,label in train_loader:
        data,label = data.cuda(),label.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output,label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss/len(train_loader.dataset)
    print('Epoch:{}\tTraining Loss:{:.6f}'.format(epoch,train_loss))

In [25]:
def val(epoch):
    model.eval()
    val_loss = 0
    gt_labels = []
    pre_labels = []

    with torch.no_grad():
        for data,label in test_loader:
            data,label = data.cuda(),label.cuda()
            output = model(data)
            preds = torch.argmax(output,1)
            gt_labels.append(label.cpu().data.numpy())
            pre_labels.append(preds.cpu().numpy())
            loss = criterion(output,label)
            val_loss += loss.item()*data.size(0)
    val_loss = val_loss/len(test_loader.dataset)
    gt_labels,pre_labels = np.concatenate(gt_labels),np.concatenate(pre_labels)
    acc = np.sum(gt_labels == pre_labels)/len(pre_labels)
    print('Epoch:{}\tValidation Loss:{:.6f},Accuracy:{:6f}'.format(epoch,val_loss,acc))


            


In [ ]:
for epoch in range(1,epochs+1):
    train(epoch)
    val(epoch)
